# Optimizing Strategy Parameters

In [ ]:
???

<table style="width:100%; height:90%">
      <tr>
    <th>Parametrize the Strategy</th>
    <th>Optimizing Limits' Parameters</th>
  </tr>
  <tr>
    <td><img src="src/07_Code_Regression Strategy Limits X.png" alt="Parametrize the Strategy" style="width:100%"></td>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%"></td>
  </tr>
</table>

## Load the model

In [1]:
import pickle

with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)
    
model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [2]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

Open        High         Low       Close   Adj Close  \
Date                                                                     
2016-12-08   61.299999   61.580002   60.840000   61.009998   55.705235   
2016-12-09   61.180000   61.990002   61.130001   61.970001   56.581772   
2016-12-12   61.820000   62.299999   61.720001   62.169998   56.764374   
2016-12-13   62.500000   63.419998   62.240002   62.980000   57.503944   
2016-12-14   63.000000   63.450001   62.529999   62.680000   57.230022   
...                ...         ...         ...         ...         ...   
2023-12-18  369.450012  373.000000  368.679993  372.649994  372.649994   
2023-12-19  371.489990  373.260010  369.839996  373.260010  373.260010   
2023-12-20  375.000000  376.029999  370.529999  370.619995  370.619995   
2023-12-21  372.559998  374.410004  370.040009  373.540009  373.540009   
2023-12-22  373.679993  375.179993  372.709991  374.579987  374.579987   

              Volume  change_tomorrow change_tomorrow_direction  
Date                                                             
2016-12-08  21220800         1.549141                        UP  
2016-12-09  27349400         0.321694                        UP  
2016-12-12  20198100         1.286125                        UP  
2016-12-13  35718900        -0.478620                      DOWN  
2016-12-14  30352700        -0.159793                      DOWN  
...              ...              ...                       ...  
2023-12-18  21802900         0.163429                        UP  
2023-12-19  20603700        -0.712324                      DOWN  
2023-12-20  26316700         0.781714                        UP  
2023-12-21  17708000         0.277638                        UP  
2023-12-22  17091100         0.021357                        UP  

[1772 rows x 8 columns]

# Simple Investment Strategy

### Create Strategy class

In [3]:
from backtesting import Strategy, Backtest

In [4]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [5]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [6]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting with specific values

In [7]:
results = bt.run()

### Interpret backtesting results

In [8]:
results.to_frame(name='Values').loc[:'Return [%]']

Values
Start              2016-12-08 00:00:00
End                2023-12-22 00:00:00
Duration            2570 days 00:00:00
Exposure Time [%]              99.8307
Equity Final [$]          55709.181907
Equity Peak [$]           57053.164817
Return [%]                  457.091819

## Parametrize the Investment Strategy

### Create Strategy class

In [9]:
from backtesting import Strategy, Backtest

In [10]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [11]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

## Optimize backtesting with multiple combinations

In [12]:
list_limits_buy = list(range(0, 11, 1))
list_limits_buy

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
list_limits_sell = list(range(0, -11, -1))
list_limits_sell

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

In [14]:
%%time

results = bt.optimize(
    limit_buy = list_limits_buy, limit_sell = list_limits_sell,
    maximize='Return [%]', return_heatmap=True
)

CPU times: user 3.39 s, sys: 22.9 ms, total: 3.41 s
Wall time: 2min 36s


In [15]:
results[0]

Start                     2016-12-08 00:00:00
End                       2023-12-22 00:00:00
Duration                   2570 days 00:00:00
Exposure Time [%]                   99.887133
Equity Final [$]               2551119.494212
Equity Peak [$]                2685823.890329
Return [%]                       25411.194942
Buy & Hold Return [%]              513.964919
Return (Ann.) [%]                  119.920876
Volatility (Ann.) [%]               59.919511
Sharpe Ratio                         2.001366
Sortino Ratio                        7.500473
Calmar Ratio                         7.128199
Max. Drawdown [%]                  -16.823447
Avg. Drawdown [%]                   -2.421032
Max. Drawdown Duration      183 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                  494
Win Rate [%]                        64.777328
Best Trade [%]                      19.537685
Worst Trade [%]                     -8.630213
Avg. Trade [%]                    

In [16]:
results[1]

limit_buy  limit_sell
0           0            25411.194942
           -1            12727.678550
           -2             5410.271273
           -3             2779.130362
           -4             1343.882491
                             ...     
10         -6             -100.000000
           -7             -100.000000
           -8             -100.000000
           -9             -100.000000
           -10            -100.000000
Name: Return [%], Length: 121, dtype: float64

### [ ] Interpret optimization results

In [17]:
dff = results[1].reset_index()
dff

limit_buy  limit_sell    Return [%]
0            0           0  25411.194942
1            0          -1  12727.678550
2            0          -2   5410.271273
3            0          -3   2779.130362
4            0          -4   1343.882491
..         ...         ...           ...
116         10          -6   -100.000000
117         10          -7   -100.000000
118         10          -8   -100.000000
119         10          -9   -100.000000
120         10         -10   -100.000000

[121 rows x 3 columns]

In [18]:
dff = dff.pivot(index='limit_buy', columns='limit_sell', values='Return [%]')

### DataFrame heatmaps for better reporting

In [19]:
dff.sort_index(axis=1, ascending=False)\
  .style.format(precision=0)\
    .background_gradient(vmin=dff.values.min(), vmax=dff.values.max())